# Binary Classification

In [1]:
import toml
import pandas as pd

from app.cleaning import LabelCleaner
from app.imaging import ImageHandler
from app.model import ModelController, ModelParameters, ModelPreparation, ModelPredictor
from app.utils import load_model
from app.plotter import plot_cm, plot_losses, plot_model_preds

In [2]:
all_params = toml.load("parameters.toml")

label_params = all_params['labelparameters']
model_params_dict = {key.lower(): val for key, val in all_params['modelparameters'].items()}

In [3]:
print(label_params)
print(model_params_dict)

{'LABEL_CLASSES': ['civilian', 'military'], 'LABEL_COLUMNS': ['filename', 'class', 'type']}
{'epochs': 1000, 'learning_rate': 0.001, 'batch_size': 128, 'hidden_layer_sizes': [512, 256], 'num_classes': 2, 'seed': 386}


## Label Pre-Processing

In [4]:
train_labels = pd.read_csv('labels/train_labels.csv', sep=',')
test_labels = pd.read_csv('labels/test_labels.csv', sep=',')

In [5]:
lb = LabelCleaner(train_labels, test_labels, labels=label_params['LABEL_CLASSES'])
data_labels = lb.clean_data()

train_labels overlap duplicates removed: 75
test_labels overlap duplicates removed: 6


In [6]:
data_labels = data_labels[label_params['LABEL_COLUMNS']]

## Update Image Names & Location

In [7]:
img_handler = ImageHandler(data_labels, class_labels=label_params['LABEL_CLASSES'])

In [8]:
img_handler.update_data_dir()

Total invalid: 6628
Moved: 0 imgs -> civilian directory, 0 imgs -> military directory


C:\Projects\Rowden\app\imaging.py:65: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len_label_a = len(self.data_labels[filepaths.str.contains(self.labels[0])])
C:\Projects\Rowden\app\imaging.py:66: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len_label_b = len(self.data_labels[filepaths.str.contains(self.labels[1])])


In [9]:
data_labels = img_handler.data_labels
data_labels

,filename,class,type,img_filepaths


In [10]:
train_indices = data_labels[data_labels['type'] == 'train'].index.to_list()
test_indices = data_labels[data_labels['type'] == 'test'].index.to_list()

In [11]:
model_params = ModelParameters(**model_params_dict)
model_prep = ModelPreparation(model_params)

In [12]:
train_loader, valid_loader, test_loader = model_prep.split_data(
    data_filepath='data',
    train_indices=train_indices,
    test_indices=test_indices
)

Training data: 0
Validation data: 0
Test data: 0


## Create & Train Model

In [13]:
model_controller = ModelController(model_params=model_params, train_loader=train_loader, valid_loader=valid_loader)

In [14]:
# model = model_controller.train(epochs=model_params.epochs, save_filepath='saved_models/mobilenetv3.pt')

## Load Model & Visualise Results

In [15]:
model_params = ModelParameters(**model_params_dict)
model = load_model(model_controller.model, filepath='saved_models/mobilenetv3.pt')

In [16]:
model_predictor = ModelPredictor(model, test_data=test_loader)

In [17]:
y_pred, y_true = model_predictor.predict(seed=model_params.seed)

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
stats = model_predictor.compute_stats(y_pred, y_true)
stats

In [ ]:
# | TP | FP |
# | FN | TN |
plot_cm(y_pred, y_true, label_params['LABEL_CLASSES'])

In [ ]:
plot_losses(model.logger.train_loss, model.logger.valid_loss)

In [ ]:
plot_model_preds(model, test_data=test_loader, class_labels=label_params['LABEL_CLASSES'])